In [62]:
# !pip install \
#   langchain_community \
#   langchain_pinecone \
#   langchain_openai \
#   unstructured \
#   langchain-text-splitters \
#   pinecone-text 

In [6]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import (
    PineconeHybridSearchRetriever)
#from pinecone.grpc import PineconeGRPC as Pinecone
#from langchain.vectorstores import PineconeVectorStore  
from langchain.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain.document_loaders import TextLoader
from langchain_core.runnables import Runnable
from pinecone import ServerlessSpec
import pandas as pd
import itertools
from uuid import uuid4
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
from dotenv import load_dotenv
import pinecone
import uuid
import os
import glob
import getpass
import pylibmagic
import hashlib
from tqdm.autonotebook import tqdm
from langchain.storage import InMemoryStore
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pinecone import Pinecone
from pinecone.grpc import PineconeGRPC as Pinecon
from typing import Sequence
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
os.environ['USER_AGENT'] = 'myagent'

In [64]:
import nltk
nltk.download('punkt')

In [65]:
load_dotenv()
# Fetch API keys from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
momento_api_key = os.getenv('MOMENTO_API_KEY')

# Set the environment variables
if openai_api_key:
    os.environ['OPENAI_API_KEY'] = openai_api_key
if pinecone_api_key:
    os.environ['PINECONE_API_KEY'] = pinecone_api_key 

#Verify that the keys are loaded
#print(f"OpenAI API Key: {os.environ.get('OPENAI_API_KEY')}")
#print(f"Pinecone API Key: {os.environ.get('PINECONE_API_KEY')}")


In [66]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Embedding model setup

In [67]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",                             #response time is 9s  #infloat/e5-base-V2 has 3.53sec response time.
)
embeddings


OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x345eeeee0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x345edcc70>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

## Pinecone Setup

In [68]:
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
import time

index_name = "test-3"  

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
#pc.list_indexes()

## Load Documents

In [69]:
# directory = 'data'

# def load_docs(directory):
#     loader = DirectoryLoader(directory)
#     docs = loader.load()
#     return docs

# docs = load_docs(directory)

In [70]:

directory = 'data'

def load_docs(directory, tenant_id):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    
    for doc in docs:
        doc.metadata['tenantId'] = tenant_id  

        doc.metadata['fileName'] = os.path.basename(doc.metadata['source'])
        print(doc.metadata['fileName'])

    return docs

tenantId = "tenant8"
docs = load_docs(directory, tenantId)

## Web Loader

In [71]:
%pip install -qU langchain_community beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [72]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://collegerecon.com/")

In [73]:
#loader_multiple_pages = WebBaseLoader("https://collegerecon.com/")

In [74]:
from langchain_community.document_loaders import WebBaseLoader

tenantId = "tenant1"

loader = WebBaseLoader("https://collegerecon.com/")

docs = loader.load()

for doc in docs:
    doc.metadata["tenantId"] = tenantId

if docs:
    print(f"Updated Metadata: {docs[0].metadata}")
else:
    print("No documents loaded.")

Updated Metadata: {'source': 'https://collegerecon.com/', 'title': 'Military Tuition Assistance, Veterans Education Benefits, VA Benefits', 'description': 'Uncover Tuition Assistance, College Education Benefits, and scholarships for Military Service Members, Veterans, Spouses, and Dependents.', 'language': 'en', 'tenantId': 'tenant1'}


In [75]:
import requests
from bs4 import BeautifulSoup

def get_all_links(base_url, depth=1):
    """Recursively fetch all links from the website up to a given depth."""
    visited = set()
    links_to_visit = {base_url}
    all_links = set()

    for _ in range(depth):  
        new_links = set()
        for url in links_to_visit:
            if url not in visited:
                try:
                    response = requests.get(url)
                    soup = BeautifulSoup(response.content, "html.parser")
                    for a_tag in soup.find_all("a", href=True):
                        href = a_tag["href"]
                        if href.startswith("/"):
                            href = base_url + href
                        if href.startswith("http"):
                            new_links.add(href)
                except Exception as e:
                    print(f"Error fetching {url}: {e}")
                visited.add(url)
        links_to_visit = new_links
        all_links.update(new_links)

    return list(all_links)

base_url = "https://collegerecon.com/"
all_links = get_all_links(base_url, depth=2)  
print(f"Discovered {len(all_links)} links.")


Discovered 0 links.


In [76]:
from langchain_community.document_loaders import WebBaseLoader

# Use the collected links
loader = WebBaseLoader(all_links)

# Load the documents
docs = loader.load()

# Example: Print the metadata and content of the first document
if docs:
    print(f"Document Metadata: {docs[0].metadata}")
    print(f"Document Content: {docs[0].page_content[:500]}...")  # Print the first 500 characters


## Generate uniqueId for Docs

In [77]:
chunk_size = 500 
chunk_overlap = 50

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

split_docs = text_splitter.split_documents(docs)

In [78]:
# import json


# split_docs = []
# chunk_ids = []

# uploaded_documents_path = "do_not_delete_uploaded_documents.json"

# if not os.path.exists(uploaded_documents_path):
#         with open(uploaded_documents_path, "w") as f:
#             json.dump({}, f)

# with open(uploaded_documents_path, "r") as f:
#         upload_documents = json.load(f)

# for doc in docs:
#         curr_split_docs = text_splitter.split_documents([doc])

#         document_id = str(uuid4())

#         # Create unique IDs for each chunk with the document ID as a prefix
#         curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]

#         split_docs = split_docs + curr_split_docs
#         chunk_ids = chunk_ids + curr_chunk_ids

#         upload_documents[document_id] =  {"fileName": doc.metadata['fileName'], "id": document_id, "tenantId": tenantId}

## Initialize Vector store

In [79]:
import json


split_docs = []
chunk_ids = []

uploaded_documents_path = "do_not_delete_uploaded_documents.json"

if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

for doc in docs:
        curr_split_docs = text_splitter.split_documents([doc])

        document_id = str(uuid4())

        # Create unique IDs for each chunk with the document ID as a prefix
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]

        split_docs = split_docs + curr_split_docs
        chunk_ids = chunk_ids + curr_chunk_ids

        upload_documents[document_id] =  {"id": document_id, "tenantId": tenantId}

In [80]:
index_name = "test-3"

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
vectorstore.add_documents(documents=split_docs, ids=chunk_ids)


[]

In [81]:
docs

[]

## qa_chain

In [82]:
# retriever = vectorstore.as_retriever(
#     search_type="similarity",
#     search_kwargs={
#         "k": 1,
#             "filter" : {
#         'source': {'$eq': 'data/xyz/Cases Module ✓.docx'}  
#     },
#             }
# )

# llm = ChatOpenAI(
#     model="gpt-4o",
#     temperature=0.0,
# )

# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=vectorstore.as_retriever(),

# )
# qa.invoke("how to issue a refund? ")

## Template

In [83]:
#query1 = "how to make a new case?"


## With and Without Knowledge Base


In [84]:
# print("Chat with knowledge:")
# print(qa.invoke(query1).get("result"))
# #print("\nChat without knowledge:")
# print(llm.invoke(query1).content)

## Filter based on built-in metadata [Source]

In [85]:
docs

[]

## Retriveal QA chain 

In [86]:
retriever = vectorstore.as_retriever(
    #search_type="similarity",
    search_kwargs={
        "k": 1,
            "filter" : {
        'tenantId': {'$eq': 'tenant7'}  
    },
            }
)

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0,
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
)

# answer = qa("How to add a new case?")
# print(answer)
answer = qa.invoke("How to issue refund?")
answer

# async for response in qa.astream({"query": "How to issue a refund?"}):
#     print(response)  


{'query': 'How to issue refund?',
 'result': 'To issue a refund, you typically need to follow the specific refund process of the company or platform you are dealing with. This may involve logging into your account, locating the transaction, and selecting the option to issue a refund. If you are a business owner, you may need to process the refund through your payment processor or point-of-sale system. If you provide more details about the specific situation or platform you are using, I may be able to provide more specific guidance.'}

## Describe Vector Database

In [87]:
pc.describe_index("test-3")


{'deletion_protection': 'enabled',
 'dimension': 1536,
 'host': 'test-3-unx28qm.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'test-3',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

## Delete by embeddedId

In [88]:
for ids in index.list(prefix='71580ad0-1042-4260-8a9b-d41a2325070e_'):
    print(ids)
    index.delete(ids=ids)

## RAG Chain Initiations

In [89]:
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

retriever = vectorstore.as_retriever(
    #search_type="similarity",
    search_kwargs={
        "k": 1,
            "filter" : {
        'tenantId': {'$eq': 'tenant7'}  
    },
            }
)

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0,
    n=1,
)

set_llm_cache(InMemoryCache())

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
)

answer = qa("How to issue refund?")
print(answer)
# answer = qa.invoke("How to add new appointment? ")
# answer


{'query': 'How to issue refund?', 'result': "I don't know."}


## RAG : Parallel Langchain Chains with Streaming

In [90]:
template = """Answer the question based only on the following context:
{context}
Question: {question}
Answer: """
retriever = vectorstore.as_retriever(
    #search_type="similarity",
    search_kwargs={
        "k": 1,
            "filter" : {
        'tenantId': {'$eq': 'tenant7'}  
    },
            }
)
prompt = ChatPromptTemplate.from_template(template)

# def docs2str(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Streaming

In [91]:
for chunk in rag_chain.stream("How to issue refund?"):
    print(chunk, end="", flush=True)

Contact customer support for assistance.

## Q/A with RAG Chain

In [92]:
question = "how to issue refund?"
response = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {response}")


Question: how to issue refund?
Answer: Contact customer support for assistance with issuing a refund.


## Add chat history


In [93]:
retriever = vectorstore.as_retriever(
    #search_type="similarity",
    search_kwargs={
        "k": 1,
            "filter" : {
        'tenantId': {'$eq': 'tenant7'}  
    },
            }
)

### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###


# We define a dict representing the state of the application.
# This state has the same input and output keys as `rag_chain`.
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str


# We then define a simple node that runs the `rag_chain`.
# The `return` values of the node update the graph state, so here we just
# update the chat history with the input message and response.
def call_model(state: State):
    response = rag_chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


# Our graph consists only of one node:
workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Finally, we compile the graph with a checkpointer object.
# This persists the state, in this case in memory.
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [94]:
config = {"configurable": {"thread_id": "a2"}}

result = app.invoke(
    {"input": "How to issue refund?"},
    config=config,
)
print(result["answer"])

To issue a refund, you typically need to access your payment processing system or platform where the original transaction took place. Locate the specific transaction that requires a refund and follow the steps provided by the platform to initiate the refund process. Make sure to confirm the refund amount and provide any necessary details before finalizing the refund transaction.


In [95]:
result = app.invoke(
    {"input": "Tell me one of the way of doing it"},
    config=config,
)
print(result["answer"])

One way to issue a refund is through your online payment processor or platform. Log in to your account, locate the transaction that needs a refund, and follow the instructions to process the refund. Ensure that you enter the correct refund amount and provide any required information before confirming the refund.


In [96]:
result = app.invoke(
    {"input": "Who should i contact to do it?"},
    config=config,
)
print(result["answer"])

/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/typing.py:292: RuntimeWarning: coroutine 'AsyncChromiumLoader.ascrape_playwright' was never awaited
  ev_args = tuple(_eval_type(a, globalns, localns, recursive_guard) for a in t.__args__)


You should contact the customer support or technical support team of the payment processing platform you use to issue a refund. They can guide you through the process, provide assistance, and address any issues you may encounter while issuing the refund. Be prepared to provide details about the transaction in question to expedite the refund process.


In [97]:
from langchain.docstore.document import Document


In [99]:
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pinecone_index_name = os.environ.get("PINECONE_INDEX_NAME")
pc = Pinecone(api_key=pinecone_api_key)
pinecone_client = Pinecone(api_key=pinecone_api_key)
def initialize_pinecone_index(client: Pinecone, index_name: str):
    existing_indexes = [index_info["name"] for index_info in client.list_indexes()]
    if index_name not in existing_indexes:
        client.create_index(
            name=index_name,
            dimension=3072,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1"))
        while not client.describe_index(index_name).status["ready"]:
            time.sleep(1)
    return client.Index(index_name)
index = initialize_pinecone_index(pinecone_client, pinecone_index_name)

In [100]:
# Define the URL to scrape
url = "https://www.noveltytechnology.com/about-us"

# Fetch the webpage content
response = requests.get(url)
if response.status_code == 200:
    html_content = response.text
else:
    raise Exception(f"Failed to retrieve the webpage. Status code: {response.status_code}")

# Parse the HTML content using Beautiful Soup
soup = BeautifulSoup(html_content, 'html.parser')

# Extract the main content of the page
page_content = "\n".join([p.get_text() for p in soup.find_all('p')])


In [101]:
# Create a Document object with the extracted content
document = Document(page_content=page_content.strip(), metadata={"source": url})

# Initialize the OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Generate embeddings for the document
doc_embeddings = embeddings.embed_documents([document.page_content])


In [104]:
# Prepare the data for upsert
# Assuming 'doc_embeddings' is a list of embeddings and 'document' has a unique ID
vectors = [(str(i), embedding) for i, embedding in enumerate(doc_embeddings)]

# Upsert the vectors into Pinecone
#index.upsert(vectors)


In [105]:
# Define your query
query = "Tell me about Novelty Technology"

# Generate the query embedding
query_embedding = embeddings.embed_query(query)

# Perform a similarity search in Pinecone
search_results = index.query(queries=[query_embedding], top_k=5)

# Process and print the results
for match in search_results['matches']:
    print(f"Score: {match['score']}")
    # Retrieve the original document content or metadata as needed


ForbiddenException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'Date': 'Thu, 02 Jan 2025 06:11:13 GMT', 'Content-Type': 'text/plain', 'Content-Length': '9', 'Connection': 'keep-alive', 'x-pinecone-auth-rejected-reason': 'Wrong API key', 'www-authenticate': 'Wrong API key', 'server': 'envoy'})
HTTP response body: Forbidden


In [37]:
load_dotenv()

apify_api_key = os.getenv("APIFY_API_TOKEN")
if apify_api_key:
    os.environ["APIFY_API_TOKEN"] = apify_api_key

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.docstore.document import Document
from langchain_community.utilities import ApifyWrapper

apify = ApifyWrapper()
# Call the Actor to obtain text from the crawled webpages
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": [{"url": "https://www.noveltytechnology.com/about-us"}]},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

# Create a vector store based on the crawled data
#index = VectorstoreIndexCreator().from_loaders([loader])


# Query the vector store
query = "Tell me about the novelty technology"
result = index.query(query, top_k=5)
print(result)

ValueError: The argument order for `query()` has changed; please use keyword arguments instead of positional arguments. Example: index.query(vector=[0.1, 0.2, 0.3], top_k=10, namespace='my_namespace')

## Final Output with message history and desired output format

In [ ]:
response = app.invoke(
    {"ability": "Vitafy System", "input": "How to issue refund?"},
    config={"configurable": {"session_id": "abc12"}},
)

if isinstance(response, AIMessage):
    print(response.content)
else:
    print("The system is under maintainance so, I recommend reaching out to your system administrator or support team for further assistance.")


KeyError: 'thread_id'

In [236]:
response = with_message_history.invoke(
    {"ability": "Vitafy System", "input": "Tell me one of the way of doing it"},
    config={"configurable": {"session_id": "abc12"}},
)

if isinstance(response, AIMessage):
    print(response.content)
else:
    print("The system is under maintainance so, I recommend reaching out to your system administrator or support team for further assistance.")


One way to issue a refund using the Vitafy System is to access the transaction details of the specific order for which you need to issue a refund. Within the transaction details, there should be an option or button to initiate a refund. Click on this option, enter the refund amount, provide any necessary details or reasons for the refund, and then confirm the refund to process it through the Vitafy System. Be sure to double-check the refund amount and details before finalizing the refund to ensure accuracy.


In [252]:
runnable_with_history = RunnableWithMessageHistory(
    model,
    get_session_history,
)

In [255]:
runnable_with_history.invoke(
    [HumanMessage(content="How to issue refund?")],
    config={"configurable": {"session_id": "1"}},
)

AIMessage(content='To issue a refund, you typically need to follow these steps:\n\n1. Log in to your payment processing platform or account.\n2. Locate the transaction for which you want to issue a refund.\n3. Click on the option to issue a refund or process a return.\n4. Enter the refund amount and any additional details required.\n5. Confirm the refund and complete the process.\n\nPlease note that the specific steps may vary depending on the payment processor or platform you are using. If you need further assistance, I recommend reaching out to the customer support of your payment processor or platform for detailed instructions.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 59, 'total_tokens': 178, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name':

In [254]:
runnable_with_history.invoke(
    [HumanMessage(content="whats my name?")],
    config={"configurable": {"session_id": "1"}},
)

AIMessage(content='Your name is Ujjwal!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 39, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-65af7fd4-dcd5-42a1-88e3-c80df40c8bf6-0', usage_metadata={'input_tokens': 39, 'output_tokens': 7, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
from operator import itemgetter
from typing import List
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field
from langchain_core.runnables import (
    RunnableLambda,
    ConfigurableFieldSpec,
    RunnablePassthrough,
)
from langchain_core.runnables.history import RunnableWithMessageHistory


class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []


# Global store to manage tenant-specific chat histories
store = {}


def get_by_tenant_and_session_id(tenant_id: str, session_id: str) -> BaseChatMessageHistory:
    """
    Retrieves or creates a tenant-specific chat message history for a given session ID.
    """
    # Use a composite key combining tenant_id and session_id
    key = f"{tenant_id}:{session_id}"
    if key not in store:
        store[key] = InMemoryHistory()
    return store[key]


# Example usage: Add messages for a specific tenant and session
tenantIdd = "tenant5"
session_id = "1"

# Retrieve history for the specific tenant and session
history = get_by_tenant_and_session_id(tenantId, session_id)
history.add_message(AIMessage(content="hello, this is a tenant-specific message!"))
print(store)  # Inspect tenant-specific chat history


# Example of using RunnableWithMessageHistory with tenant-specific history
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {ability}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain = prompt | ChatOpenAI(model="gpt-3.5-turbo")

chain_with_history = RunnableWithMessageHistory(
    chain,
    # Use the tenant-specific retrieval function
    lambda session_id: get_by_tenant_and_session_id(tenant_id, session_id),
    input_messages_key="question",
    history_messages_key="history",
)

# Simulate a conversation for a specific tenant and session
response = chain_with_history.invoke(
    {"ability": "   Vitafy AI", "question": "One way of doing refund"},
    config={"configurable": {"session_id": session_id}}
)
print(response)  # Output for the question
print(store)  # Inspect the store to see the tenant-specific history


{'tenant5:1': InMemoryHistory(messages=[AIMessage(content='hello, this is a tenant-specific message!', additional_kwargs={}, response_metadata={})])}
content="One way to process a refund is to follow these steps:\n\n1. Confirm the reason for the refund request and verify the details of the transaction.\n2. Access the payment system or platform used for the original transaction.\n3. Locate the transaction in question and select the option to issue a refund.\n4. Enter the refund amount and any necessary details or comments.\n5. Process the refund through the system, which will typically notify the customer of the refund.\n6. Confirm with the customer that the refund has been processed and provide any additional information or assistance as needed.\n\nIt's important to ensure accurate and timely processing of refunds to maintain customer satisfaction." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 129, 'prompt_tokens': 42, 'total_tokens': 171,

In [237]:
# response_stream = with_message_history.astream(
#     {"ability": "Vitafy System", "input": "How to contact them?"},
#     config={"configurable": {"session_id": "abc12"}},
# )

# # Stream and print only the content
# async for chunk in response_stream:
#     if "content" in chunk:
#         print(chunk["content"], end="", flush=True)

# if isinstance(response, AIMessage):
#     print(response.content)
# else:
#     print("Unexpected response format")

In [263]:
from typing import List, Dict
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import StrOutputParser

# In-memory tenant-specific chat history
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store."""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

# Tenant-specific history store
store = {}

def get_by_tenant_and_session_id(tenant_id: str, session_id: str) -> BaseChatMessageHistory:
    """
    Retrieves or creates a tenant-specific chat message history for a given session ID.
    """
    key = f"{tenant_id}:{session_id}"
    if key not in store:
        store[key] = InMemoryHistory()
    return store[key]

# Initialize Pinecone VectorStore
def initialize_vectorstore():
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name="test-index", embedding=embeddings)
    return vectorstore

# Define RAG Chain
def create_rag_chain(vectorstore, tenant_id: str, llm):
    template = """Answer the question based only on the following context:
    {context}
    Question: {question}
    Answer: """

    retriever = vectorstore.as_retriever(
        search_kwargs={
            "k": 1,
            "filter": {
                "tenantId": {"$eq": tenant_id}
            },
        }
    )

    prompt = ChatPromptTemplate.from_template(template)

    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return rag_chain

# Example of using RAG Chain with tenant-specific message history
def main(tenant_id: str, session_id: str, question: str):
    # Retrieve tenant-specific history
    history = get_by_tenant_and_session_id(tenant_id, session_id)

    # Initialize VectorStore
    vectorstore = initialize_vectorstore()

    # Initialize LLM
    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.2,
    )

    # Create RAG Chain
    rag_chain = create_rag_chain(retriever, tenantId, llm)

    # Define the prompt with message history
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an assistant for tenant-specific queries."),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{question}"),
        ]
    )

    # Combine RAG Chain and message history
    chain_with_history = RunnableWithMessageHistory(
        rag_chain,
        lambda session_id: get_by_tenant_and_session_id(tenant_id, session_id),
        input_messages_key="question",
        history_messages_key="history",
    )

    # Process the question with the RAG chain
    response = chain_with_history.invoke(
        {"question": question},
        config={"configurable": {"session_id": session_id}}
    )

    # Add message history
    history.add_messages([AIMessage(content=response)])

    return response


ImportError: cannot import name 'StrOutputParser' from 'langchain_core.prompts' (/Users/ujjwal/Library/Python/3.9/lib/python/site-packages/langchain_core/prompts/__init__.py)

In [246]:
from datetime import timedelta
from momento import CacheClient, Configurations, CredentialProvider
from momento.responses import CacheGet, CacheSet, CreateCache, ListCaches, CacheIncrement

import os

def create_client():
  momento_api_key = CredentialProvider.from_environment_variable('MOMENTO_API_KEY')
  ttl  = timedelta(seconds=int(os.getenv('MOMENTO_TTL_SECONDS', '600')))
  config = {
    'configuration': Configurations.Laptop.v1(),
    'credential_provider': momento_api_key,
    'default_ttl':  ttl
  }
  return CacheClient.create(**config)

In [249]:
def create_cache(client, cache_name: str) -> None:
    resp = client.create_cache(cache_name)
    
    if isinstance(resp, CreateCache.Success):
        print("Cache created.")
    elif isinstance(resp, CreateCache.Error):
        print(f"Error creating cache: {resp.message}")
    else:
        print(f"Unreachable with {resp.message}")


In [250]:
from datetime import timedelta

from langchain_community.chat_message_histories import MomentoChatMessageHistory

session_id = "foo"
cache_name = "langchain"
ttl = timedelta(days=1)
history = MomentoChatMessageHistory.from_client_params(
    session_id,
    cache_name,
    ttl,
)

history.add_user_message("hi!")

history.add_ai_message("whats up?")

PermissionDeniedException: PermissionDeniedException(message='Unauthorized: Token is not authorized to make this call', error_code=<MomentoErrorCode.PERMISSION_ERROR: (6,)>, transport_details=MomentoErrorTransportDetails(grpc=MomentoGrpcErrorDetails(code=<StatusCode.PERMISSION_DENIED: (7, 'permission denied')>, details='Unauthorized: Token is not authorized to make this call', metadata=Metadata((('err', 'momento_general_err'), ('content-length', '0'), ('vary', 'origin, access-control-request-method, access-control-request-headers'), ('access-control-allow-credentials', 'true'), ('access-control-expose-headers', 'grpc-status,grpc-message,grpc-encoding,grpc-accept-encoding,err'), ('date', 'Thu, 28 Nov 2024 00:18:07 GMT'))))), message_wrapper='Insufficient permissions to perform an operation on the cache')

## Redis

In [279]:
%pip install -qU langchain-redis langchain-openai redis

Note: you may need to restart the kernel to use updated packages.


In [280]:
if os.environ.get("ENV") == "production":
    REDIS_URL = "redis://production-redis-server:6379"
elif os.environ.get("ENV") == "staging":
    REDIS_URL = "redis://staging-redis-server:6379"

print(f"Connecting to Redis at: {REDIS_URL}")

Connecting to Redis at: redis://localhost:6379


In [281]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_redis import RedisChatMessageHistory

In [282]:
# Initialize RedisChatMessageHistory
history = RedisChatMessageHistory(session_id="user123", redis_url=REDIS_URL)

# Add messages to the history
history.add_user_message("Hello, AI assistant!")
history.add_ai_message("Hello! How can I assist you today?")

# Retrieve messages
print("Chat History:")
for message in history.messages:
    print(f"{type(message).__name__}: {message.content}")

ResponseError: unknown command 'FT.INFO', with args beginning with: 'idx:chat_history' 

In [ ]:
import uuid
import pylibmagic
import io
import pinecone
import shutil
import uvicorn
import numpy as np
import os
import pinecone
import glob
import time
import getpass
import concurrent.futures
import docx
import json

from tqdm.auto import tqdm
from fastapi import  Form
from uuid import uuid4
from typing import List, Dict
from typing import Any
from fastapi.responses import JSONResponse
from PyPDF2 import PdfReader  
from fastapi import Form, BackgroundTasks
from langchain_community.document_loaders import WebBaseLoader
from fastapi.responses import StreamingResponse
from fastapi.responses import PlainTextResponse
from fastapi.middleware.cors import CORSMiddleware
from fastapi import Request
from fastapi import BackgroundTasks
from fastapi import Query
from langchain_core.documents import Document
from dotenv import load_dotenv
from fastapi.responses import RedirectResponse
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langserve import add_routes
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from langchain_core.runnables import RunnableParallel
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.runnables import Runnable
from pydantic import BaseModel, Field, validator, ValidationError
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages.base import BaseMessage
from fastapi import UploadFile, File, HTTPException
from langchain_openai import OpenAIEmbeddings
from langchain_community.retrievers import PineconeHybridSearchRetriever

class MyModel(BaseModel):
    message: BaseMessage


class Config:
    arbitrary_types_allowed = True


app = FastAPI(
    title="LangChain Server",
    version="o1",
    description="",
)
# Set all CORS enabled origins
origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    start_time = time.perf_counter()
    response = await call_next(request)
    process_time = time.perf_counter() - start_time
    response.headers["Processing-Time"] = str(process_time)
    return response


load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = os.getenv("PINECONE_INDEX_NAME")
# langchain_tracing = os.getenv("LANGCHAIN_TRACING_V2")
# langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
# langchain_project = os.getenv("LANGCHAIN_PROJECT")
# langchain_endpoint = os.getenv("LANGCHAIN_ENDPOINT")

if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
if pinecone_api_key:
    os.environ["PINECONE_API_KEY"] = pinecone_api_key
if pinecone_index_name:
    os.environ["PINECONE_INDEX_NAME"] = pinecone_index_name
# if langchain_tracing:
#     os.environ["LANGCHAIN_TRACING_V2"] = langchain_tracing
# if langchain_api_key:
#     os.environ["LANGCHAIN_API_KEY"] = langchain_api_key  
# if langchain_project:
#     os.environ["LANGCHAIN_PROJECT"] = langchain_project    
# if langchain_endpoint:
#     os.environ["LANGCHAIN_ENDPOINT"] = langchain_endpoint             

# Verify that the keys are loaded
# print(f"OpenAI API Key: {os.environ.get('OPENAI_API_KEY')}")
# print(f"Pinecone API Key: {os.environ.get('PINECONE_API_KEY')}")
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")


pinecone_api_key = os.environ.get("PINECONE_API_KEY")
index_name = os.environ.get("PINECONE_INDEX_NAME")

pc = Pinecone(api_key=pinecone_api_key)

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

############################# upload #############################

############################# upload #############################

############################# upload #############################
@app.post("/api/artificial-intelligence/upload")
async def upload_files(
    tenantId: str = Form(...),
    files: List[UploadFile] = File(...),
    background_task: BackgroundTasks = BackgroundTasks(),
):
    """Upload multiple PDF, DOCX, and TXT files"""
    dir_name = str(uuid4())

    os.makedirs(dir_name, exist_ok=True)

    # Allowed file extensions
    allowed_extensions = {".pdf", ".docx", ".txt"}

    # Tracking file names to check for duplicates
    fileName = set()

    # Saving each uploaded file
    for file in files:
        if file.filename in fileName:
            raise HTTPException(
                status_code=400, detail=f"Duplicate file detected: {file.filename}"
            )

        fileName.add(file.filename)

        # Check for file extension
        _, extension = os.path.splitext(file.filename)
        if extension.lower() not in allowed_extensions:
            raise HTTPException(
                status_code=400,
                detail=f"Invalid file type: {file.filename}. Only PDF, DOCX, and TXT files are allowed.",
            )

        # Define the destination path for the uploaded file
        destination = os.path.join(dir_name, file.filename)

        print('creating doc ' + destination)
        # Save the uploaded file to the tenant's directory
        with open(destination, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)



    # Load the documents from the tenant's directory
    docs = load_docs(dir_name, tenantId)
    embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002", #response time is 9s  #infloat/e5-base-V2 has 3.53sec response time.
    )
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)


    chunk_size = 1000 
    chunk_overlap = 20

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    split_docs = []
    chunk_ids = []

    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    for doc in docs:
        curr_split_docs = text_splitter.split_documents([doc])

         # Generate a unique document ID
        document_id = str(uuid4())

        # Create unique IDs for each chunk with the document ID as a prefix
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]

        split_docs = split_docs + curr_split_docs
        chunk_ids = chunk_ids + curr_chunk_ids

        upload_documents[document_id] =  {"fileName": doc.metadata['filename'], "id": document_id, "tenantId": tenantId}

    # Add documents to vector store with unique chunk IDs
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)

    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)

    shutil.rmtree(dir_name)

    return {"status": "success", "message": "Files uploaded successfully."}

    

def load_docs(directory, tenantId):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    
    for doc in docs:
        doc.metadata['tenantId'] = tenantId

        doc.metadata['filename'] = os.path.basename(doc.metadata['source'])
        print(doc.metadata['filename'])

    return docs

############################# Web-URL #############################

############################# Web-URL #############################

############################# Web-URL #############################
@app.post("/api/artificial-intelligence/websites")
async def upload_web_urls(
    tenantId: str = Form(...),
    urls: List[str] = Form(...),
):
    """
    Upload and process web URLs, then store content in Pinecone.
    """
    all_docs = []  

    for url in urls:
        try:
            loader = WebBaseLoader(url)
            docs = loader.load()  
            
            for doc in docs:
                doc.metadata["tenantId"] = tenantId
                doc.metadata["source"] = url
            all_docs.extend(docs)
        except Exception as e:
            print(f"Error loading URL {url}: {e}")
            continue

    if not all_docs:
        raise HTTPException(status_code=400, detail="No valid URLs provided or failed to fetch content.")

    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)


    chunk_size = 1000
    chunk_overlap = 20
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    for doc in all_docs:
        curr_split_docs = text_splitter.split_documents([doc])

        # Generate a unique document ID
        document_id = str(uuid.uuid4())

        # Create unique IDs for each chunk with the document ID as a prefix
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]

        split_docs.extend(curr_split_docs)
        chunk_ids.extend(curr_chunk_ids)

        upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": tenantId}

    # Add documents to vector store with unique chunk IDs
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)

    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)

    return {"status": "success", "message": "URLs processed and stored successfully."}

############################# Add Q&A #############################

############################# Add Q&A #############################

############################# Add Q&A #############################
@app.post("/api/artificial-intelligence/Add Q&A")
async def upload_question_answer(
    tenantId: str = Form(...),
    question: List[str] = Form(...),
    answer: List[str] = Form(...),
    background_task: BackgroundTasks = BackgroundTasks(),
):
    """
    Upload and process a question and answer pair and store it in Pinecone.
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

    chunk_size = 1000
    chunk_overlap = 20
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    split_docs = []
    chunk_ids = []

    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    try:
        document_id = str(uuid.uuid4())

        combined_text = f"Question: {question}\nAnswer: {answer}"

        document_id = str(uuid.uuid4())

        metadata = {
            "tenantId": tenantId,
            "fileName": f"Q&A_input_{document_id}.txt",
            "id": document_id,
            "question": question[0],  
            "answer": answer[0], 

        }

        document = Document(page_content=combined_text, metadata=metadata)

        curr_split_docs = text_splitter.split_documents([document])


        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]

        split_docs += curr_split_docs
        chunk_ids += curr_chunk_ids

        upload_documents[document_id] = metadata
        

        # Add to Pinecone vector store
        vectorstore.add_documents(documents=split_docs, ids=chunk_ids)

        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)

        #return {"status": "success", "message": "Question and Answer processed and stored successfully."}
        return {
            "status": "success",
            "message": "Question and Answer processed  successfully.",
            "id": document_id,
            "question": question[0],  
            "answer": answer[0]       
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing the question and answer: {str(e)}")
 
############################# tenant-files #############################

############################# tenant-files #############################

############################# tenant-files #############################
@app.get("/api/artificial-intelligence/tenant_files")
async def retrieve_files(tenantId: str = Query(...)):
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    tenantFiles = []
    for item in list(upload_documents.values()):
        if item.get('tenantId') == tenantId:
            tenantFiles.append(item)

    return {"data": tenantFiles}

############################# prompt #############################

############################# prompt #############################

############################# prompt #############################
def initializeVectorStore():
    embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",                            
    )
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    return vectorstore

llm = ChatOpenAI(
    #model="gpt-4o",
    model="gpt-3.5-turbo",
    temperature=0.0,
)
@app.get("/api/artificial-intelligence/prompts")
async def prompts_keyword(tenantId: str = Query(...), keyword: str = Query(...)):
    try:
        print(tenantId, keyword)
        vectorStore = initializeVectorStore()
        retriever = vectorStore.as_retriever(
    
    search_kwargs={
        "k": 1,
            "filter" : {
        'tenantId': {'$eq': tenantId}  
    
    },
            }
)      
        newQa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=False,
        )



        answer = newQa({"query": keyword})
    #For JSON Output
    #     return {"answer": answer}
    # except Exception as e:
    #     return {"error": str(e)}
    
    #For Plain Text Output
        return PlainTextResponse(answer["result"]) 

    except Exception as e:
        return PlainTextResponse(
            f"Error: {str(e)}. Please contact the support team for assistance."
        )

############################# Delete #############################

############################# Delete #############################

############################# Delete #############################
class DeleteRequest(BaseModel):
    prefix: str
@app.delete("/api/artificial-intelligence/delete", summary="Delete documents by embeddedId", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """
    Deletes all document IDs in the Pinecone index that start with the given prefix.

    Parameters:
    - **prefix**: The prefix string to filter and delete documents by.

    Returns:
    - A JSON response with a success message or an error if no IDs were found with the given prefix.
    """
    prefix = request.prefix
    # List all IDs with the given prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    
    print(ids_to_delete)

    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No documents found with the given prefix.")

    # Delete the IDs from the index
    index.delete(ids=ids_to_delete)

    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    if prefix in upload_documents:
        del upload_documents[prefix]
    
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)

    return {"message": f"Deleted {len(ids_to_delete)} documents with prefix '{prefix}'."}





if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)







###### Dev #####
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
#from langserve import add_routes
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages.base import BaseMessage
from fastapi import UploadFile, File, HTTPException
from langchain_openai import OpenAIEmbeddings
from typing import Any
import uuid
import pylibmagic
from fastapi import Form, BackgroundTasks
from fastapi import Query
import io
import pinecone
from PyPDF2 import PdfReader
import docx
import shutil
import uvicorn
from typing import List, Dict
import numpy as np
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import ServerlessSpec
# from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import Pinecone, ServerlessSpec
# from langchain.document_loaders import DirectoryLoader
from langchain_core.runnables import RunnableParallel
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable
from pydantic import BaseModel, Field, validator, ValidationError
from tqdm.auto import tqdm
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from dotenv import load_dotenv
from fastapi.responses import RedirectResponse, JSONResponse
import pinecone
import glob
from fastapi import FastAPI, File, UploadFile, HTTPException, Form
from fastapi.responses import JSONResponse
from uuid import uuid4
import time
from fastapi.middleware.cors import CORSMiddleware
from fastapi import FastAPI, Request
from fastapi import FastAPI, BackgroundTasks, UploadFile, File, Form, HTTPException
import getpass
import os
import concurrent.futures
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import JSONResponse
from PyPDF2 import PdfReader  # For reading PDF files
import docx
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json

class MyModel(BaseModel):
    message: BaseMessage


class Config:
    arbitrary_types_allowed = True


app = FastAPI(
    title="LangChain Server",
    version="o1",
    description="",
)
# Set all CORS enabled origins
origins = ["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    start_time = time.perf_counter()
    response = await call_next(request)
    process_time = time.perf_counter() - start_time
    response.headers["Processing-Time"] = str(process_time)
    return response


load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = os.getenv("PINECONE_INDEX_NAME")




if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
if pinecone_api_key:
    os.environ["PINECONE_API_KEY"] = pinecone_api_key
if pinecone_index_name:
    os.environ["PINECONE_INDEX_NAME"] = pinecone_index_name

# Verify that the keys are loaded
# print(f"OpenAI API Key: {os.environ.get('OPENAI_API_KEY')}")
# print(f"Pinecone API Key: {os.environ.get('PINECONE_API_KEY')}")
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")


pinecone_api_key = os.environ.get("PINECONE_API_KEY")
index_name = os.environ.get("PINECONE_INDEX_NAME")

pc = Pinecone(api_key=pinecone_api_key)
import time

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)




@app.post("/api/artificial-intelligence/upload")
async def upload_files(
    tenantId: str = Form(...),
    files: List[UploadFile] = File(...),
    background_task: BackgroundTasks = BackgroundTasks(),
):
    """Upload multiple PDF, DOCX, and TXT files"""
    dir_name = str(uuid4())

    os.makedirs(dir_name, exist_ok=True)

    # Allowed file extensions
    allowed_extensions = {".pdf", ".docx", ".txt"}

    # Tracking file names to check for duplicates
    fileName = set()

    # Saving each uploaded file
    for file in files:
        if file.filename in fileName:
            raise HTTPException(
                status_code=400, detail=f"Duplicate file detected: {file.filename}"
            )

        fileName.add(file.filename)

        # Check for file extension
        _, extension = os.path.splitext(file.filename)
        if extension.lower() not in allowed_extensions:
            raise HTTPException(
                status_code=400,
                detail=f"Invalid file type: {file.filename}. Only PDF, DOCX, and TXT files are allowed.",
            )

        # Define the destination path for the uploaded file
        destination = os.path.join(dir_name, file.filename)

        print('creating doc ' + destination)
        # Save the uploaded file to the tenant's directory
        with open(destination, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)



    # Load the documents from the tenant's directory
    docs = load_docs(dir_name, tenantId)
    embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002", #response time is 9s  #infloat/e5-base-V2 has 3.53sec response time.
    )
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

    chunk_size = 1000 
    chunk_overlap = 20

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    split_docs = []
    chunk_ids = []

    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    for doc in docs:
        curr_split_docs = text_splitter.split_documents([doc])

         # Generate a unique document ID
        document_id = str(uuid4())

        # Create unique IDs for each chunk with the document ID as a prefix
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]

        split_docs = split_docs + curr_split_docs
        chunk_ids = chunk_ids + curr_chunk_ids

        upload_documents[document_id] =  {"fileName": doc.metadata['filename'], "id": document_id, "tenantId": tenantId}

    # Add documents to vector store with unique chunk IDs
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)

    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)

    shutil.rmtree(dir_name)

    return {"status": "success", "message": "Files uploaded successfully."}

    

def load_docs(directory, tenantId):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    
    for doc in docs:
        doc.metadata['tenantId'] = tenantId

        # Extract the filename from the 'source' path
        doc.metadata['filename'] = os.path.basename(doc.metadata['source'])
        print("ooooooooo" + doc.metadata['filename'])

    return docs





############################# Web-URL #############################

############################# Web-URL #############################

############################# Web-URL #############################
@app.post("/api/artificial-intelligence/links")
async def upload_web_urls(
    tenantId: str = Form(...),
    urls: List[str] = Form(...),
):
    """
    Upload and process web URLs, then store content in Pinecone.
    """
    all_docs = []  

    for url in urls:
        try:
            loader = WebBaseLoader(url)
            docs = loader.load()  
            
            for doc in docs:
                doc.metadata["tenantId"] = tenantId
                doc.metadata["source"] = url
            all_docs.extend(docs)
        except Exception as e:
            print(f"Error loading URL {url}: {e}")
            continue

    if not all_docs:
        raise HTTPException(status_code=400, detail="No valid URLs provided or failed to fetch content.")

    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)


    chunk_size = 1000
    chunk_overlap = 20
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    for doc in all_docs:
        curr_split_docs = text_splitter.split_documents([doc])

        # Generate a unique document ID
        document_id = str(uuid.uuid4())

        # Create unique IDs for each chunk with the document ID as a prefix
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]

        split_docs.extend(curr_split_docs)
        chunk_ids.extend(curr_chunk_ids)

        upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": tenantId}

    # Add documents to vector store with unique chunk IDs
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)

    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)

    return {"status": "success", "message": "URLs processed and stored successfully."}

############################# Add Q&A #############################

############################# Add Q&A #############################

############################# Add Q&A #############################
@app.post("/api/artificial-intelligence/qa")
async def upload_question_answer(
    tenantId: str = Form(...),
    question: List[str] = Form(...),
    answer: List[str] = Form(...),
    background_task: BackgroundTasks = BackgroundTasks(),
):
    """
    Upload and process a question and answer pair and store it in Pinecone.
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

    chunk_size = 1000
    chunk_overlap = 20
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    split_docs = []
    chunk_ids = []

    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    try:
        document_id = str(uuid.uuid4())

        combined_text = f"Question: {question}\nAnswer: {answer}"

        document_id = str(uuid.uuid4())

        metadata = {
            "tenantId": tenantId,
            "fileName": f"Q&A_input_{document_id}.txt",
            "id": document_id,
            "question": question[0],  
            "answer": answer[0], 

        }

        document = Document(page_content=combined_text, metadata=metadata)

        curr_split_docs = text_splitter.split_documents([document])


        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]

        split_docs += curr_split_docs
        chunk_ids += curr_chunk_ids

        upload_documents[document_id] = metadata
        

        # Add to Pinecone vector store
        vectorstore.add_documents(documents=split_docs, ids=chunk_ids)

        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)

        #return {"status": "success", "message": "Question and Answer processed and stored successfully."}
        return {
            "status": "success",
            "message": "Question and Answer processed  successfully.",
            "id": document_id,
            "question": question[0],  
            "answer": answer[0]       
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing the question and answer: {str(e)}")



@app.get("/api/artificial-intelligence/tenant_files")
async def retrieve_files(tenantId: str = Query(...)):
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    tenantFiles = []
    for item in list(upload_documents.values()):
        if item.get('tenantId') == tenantId:
            tenantFiles.append(item)

    return {"data": tenantFiles}



def initializeVectorStore():
    embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",                             #response time is 9s  #infloat/e5-base-V2 has 3.53sec response time.
    )
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    return vectorstore

llm = ChatOpenAI(
    #model="gpt-4o",
    model="gpt-3.5-turbo",
    temperature=0.0,
)
@app.get("/api/artificial-intelligence/prompts")
async def prompts_keyword(tenantId: str = Query(...), keyword: str = Query(...)):
    try:
        print(tenantId, keyword)
        vectorStore = initializeVectorStore()
        #retriever = vectorStore.as_retriever()
        retriever = vectorStore.as_retriever(
    
    #search_type="similarity",
    search_kwargs={
        "k": 1,
            "filter" : {
        'tenantId': {'$eq': tenantId}  
    
    },
            }
)
        newQa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            #retriever=vectorStore.as_retriever(),
            return_source_documents=True,
        )

        #answer = newQa.invoke({"query": keyword})
        answer = newQa({"query": keyword})


        return {"answer": answer}
    except Exception as e:
        return {"error": str(e)}




class DeleteRequest(BaseModel):
    prefix: str
@app.delete("/api/artificial-intelligence/delete", summary="Delete documents by prefix", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """
    Deletes all document IDs in the Pinecone index that start with the given prefix.

    Parameters:
    - **prefix**: The prefix string to filter and delete documents by.

    Returns:
    - A JSON response with a success message or an error if no IDs were found with the given prefix.
    """
    prefix = request.prefix
    # List all IDs with the given prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    
    print(ids_to_delete)

    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No documents found with the given prefix.")

    # Delete the IDs from the index
    index.delete(ids=ids_to_delete)

    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    if prefix in upload_documents:
        del upload_documents[prefix]
    
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)

    return {"message": f"Deleted {len(ids_to_delete)} documents with prefix '{prefix}'."}








class DeleteRequest(BaseModel):
    prefix: str
@app.delete("/api/artificial-intelligence/qa", summary="Delete QA by prefix", description="Delete all documents in the Pinecone index that match the given prefix.")
async def delete_documents(request: DeleteRequest):
    """
    Deletes all QA IDs in the Pinecone index that start with the given prefix.

    Parameters:
    - **prefix**: The prefix string to filter and delete QA by.

    Returns:
    - A JSON response with a success message or an error if no IDs were found with the given prefix.
    """
    prefix = request.prefix
    # List all IDs with the given prefix
    ids_to_delete = [id for id in index.list(prefix=prefix)]
    
    print(ids_to_delete)

    if not ids_to_delete:
        raise HTTPException(status_code=404, detail="No QA found with the given prefix.")

    # Delete the IDs from the index
    index.delete(ids=ids_to_delete)

    uploaded_documents_path = "do_not_delete_uploaded_documents.json"

    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)

    if prefix in upload_documents:
        del upload_documents[prefix]
    
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)

    return {"message": f"Deleted {len(ids_to_delete)} QA with prefix '{prefix}'."}




if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)



In [ ]:

def initializeVectorStore():
    """Initialize and configure vector store with embeddings
    Returns:
        Configured PineconeVectorStore instance
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    return vectorstore

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0
)

@app.get("/api/artificial-intelligence/prompts")
async def prompts_keyword(tenantId: str = Query(...), keyword: str = Query(...)):
    """Process keyword prompts and retrieve answers using improved RAG chain
    Args:
        tenantId: Identifier for the tenant
        keyword: Search keyword/prompt  
    Returns:
        PlainText response with answer or error message
    """
    try:
        print(tenantId, keyword)
        vectorstore = initializeVectorStore()
        
        # Define the template for more structured responses
        template = """Answer the question based only on the following context:
        {context}
        Question: {question}
        Answer: Let me help you with that based on the available information.
        """
        
        # Configure retriever with tenant filter
        retriever = vectorstore.as_retriever(
            search_kwargs={
                "k": 1,
                "filter": {
                    'tenantId': {'$eq': tenantId}
                }
            }
        )
        
        # Create prompt template
        prompt = ChatPromptTemplate.from_template(template)
        
        # Build the RAG chain
        rag_chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )
        
        # Execute the chain
        answer = rag_chain.invoke(keyword)
        
        return PlainTextResponse(answer)
        
    except Exception as e:
        error_msg = f"Error processing your request: {str(e)}. Please contact support for assistance."
        print(f"Error in prompts_keyword: {str(e)}")  # For logging
        return PlainTextResponse(error_msg, status_code=500)

In [ ]:
# def initializeVectorStore():
#     """Initialize and configure vector store with embeddings
#     Returns:
#         Configured PineconeVectorStore instance
#     """
#     embeddings = OpenAIEmbeddings(
#     model="text-embedding-ada-002")
#     vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
#     return vectorstore
# llm = ChatOpenAI(
#     model="gpt-3.5-turbo",
#     temperature=0.0)
# @app.get("/api/artificial-intelligence/prompts")
# async def prompts_keyword(tenantId: str = Query(...), keyword: str = Query(...)):
#     """Process keyword prompts and retrieve answers
#     Args:
#         tenantId: Identifier for the tenant
#         keyword: Search keyword/prompt  
#     Returns:
#         PlainText response with answer or error message
#     """
#     try:
#         print(tenantId, keyword)
#         vectorStore = initializeVectorStore()
#         retriever = vectorStore.as_retriever(
#         search_kwargs={
#         "k": 1,
#             "filter" : {
#         'tenantId': {'$eq': tenantId} }} )
#         newQa = RetrievalQA.from_chain_type(
#             llm=llm,
#             chain_type="stuff",
#             retriever=retriever,
#             return_source_documents=False)
#         answer = newQa({"query": keyword})
#     #     return PlainTextResponse(answer["result"]) 
#     # except Exception as e:
#     #     return PlainTextResponse(
#     #         f"Error: {str(e)}. Please contact the support team for assistance.")
#         if not answer["result"] or "I don't know" in answer["result"]:
#             return PlainTextResponse(
#                 "It might be helpful to consult a subject-matter expert  or refer to specific documentation or references for more detailed information."
#             )

#         return PlainTextResponse(answer["result"])

#     except Exception as e:
#         return PlainTextResponse(
#             f"Error: {str(e)}. Please contact the support team for assistance.")

def initializeVectorStore():
    """Initialize and configure vector store with embeddings
    Returns:
        Configured PineconeVectorStore instance
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    return vectorstore

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0
)
@app.get("/api/artificial-intelligence/prompts")
async def prompts_keyword(tenantId: str = Query(...), keyword: str = Query(...)):
    """Process keyword prompts and retrieve answers using improved RAG chain
    Args:
        tenantId: Identifier for the tenant
        keyword: Search keyword/prompt  
    Returns:
        PlainText response with answer or error message
    """
    try:
        print(tenantId, keyword)
        vectorstore = initializeVectorStore()
        template = """Answer the question based only on the following context:
        {context}
        Question: {question}
        Answer: Answer the same question given by the user during input in /api/artificial-intelligence/qa
        If answer is not available then say: I don't have the information you're looking for, please provide additional details.
        In response give only answer but not question.
        """
        retriever = vectorstore.as_retriever(
            search_kwargs={
                "k": 1,
                "filter": {
                    'tenantId': {'$eq': tenantId}
                }
            }
        )
        prompt = ChatPromptTemplate.from_template(template)
        rag_chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser())
        answer = rag_chain.invoke(keyword)
        return PlainTextResponse(answer)
    except Exception as e:
        error_msg = f"Error processing your request: {str(e)}. Please contact support for assistance."
        print(f"Error in prompts_keyword: {str(e)}") 
        return PlainTextResponse(error_msg, status_code=500)

In [ ]:
# def initializeVectorStore():
#     """Initialize and configure vector store with embeddings
#     Returns:
#         Configured PineconeVectorStore instance
#     """
#     embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
#     vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
#     return vectorstore

# # Contextualization 
# contextualize_q_system_prompt = (
#     "Given a chat history and the latest user question "
#     "which might reference context in the chat history, "
#     "formulate a standalone question which can be understood "
#     "without the chat history. Do NOT answer the question, "
#     "just reformulate it if needed and otherwise return it as is."
# )

# contextualize_q_prompt = ChatPromptTemplate.from_messages([
#     ("system", contextualize_q_system_prompt),
#     MessagesPlaceholder("chat_history"),
#     ("human", "{input}"),
# ])

# qa_prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful Vitafy AI assistant. Use the following context to answer the user's question."),
#     ("system", "Context: {context}"),
#     MessagesPlaceholder(variable_name="chat_history"),
#     ("human", "{input}")
# ])

# # Session Mgmt
# class ChatSessionManager:
#     _sessions: Dict[str, Dict[str, Any]] = {}

#     @classmethod
#     def get_or_create_session(cls, tenantId: str, session_id: str = None):
#         """
#         Get or create a conversation session
        
#         Args:
#             tenantId (str): Tenant identifier
#             session_id (str, optional): Existing session ID
        
#         Returns:
#             dict: Session state
#         """
#         if not session_id:
#             session_id = str(uuid.uuid4())
        
#         if session_id not in cls._sessions:
#             cls._sessions[session_id] = {
#                 "tenantId": tenantId,
#                 "chat_history": [],
#                 "context": ""
#             }
        
#         return session_id, cls._sessions[session_id]

#     @classmethod
#     def update_session(cls, session_id: str, user_input: str, ai_response: str):
#         """
#         Update session with new interaction
        
#         Args:
#             session_id (str): Session identifier
#             user_input (str): User's message
#             ai_response (str): AI's response
#         """
#         if session_id in cls._sessions:
#             cls._sessions[session_id]["chat_history"].extend([
#                 {"role": "human", "content": user_input},
#                 {"role": "ai", "content": ai_response}
#             ])

# @app.get("/api/artificial-intelligence/prompts")
# async def prompts_keyword(
#     tenantId: str = Query(...), 
#     keyword: str = Query(...),
#     session_id: str = Query(default=None)
# ):
#     """Process keyword prompts with enhanced contextual awareness"""
#     try:
#         #  sessions mgmt
#         session_id, session_state = ChatSessionManager.get_or_create_session(
#             tenantId=tenantId, 
#             session_id=session_id
#         )
        
#         vectorstore = initializeVectorStore()
#         retriever = vectorstore.as_retriever(
#             search_kwargs={
#                 "k": 3,  
#                 "filter": {
#                     'tenantId': {'$eq': tenantId}
#                 }
#             }
#         )
        
#         # chat history for context
#         chat_history = [
#             HumanMessage(content=msg['content']) if msg['role'] == 'human' 
#             else AIMessage(content=msg['content']) 
#             for msg in session_state.get('chat_history', [])
#         ]
        
#         # LLM 
#         llm = ChatOpenAI(
#             model="gpt-3.5-turbo",
#             temperature=0.0,
#             #max_tokens=150,  # Limit response length
#             top_p=1.0,  # Nucleus sampling
#             frequency_penalty=0.5,  # Reduce repetition
#             stop=["\n"]) #stop sequence
        
#         # history-aware retriever
#         history_aware_retriever = create_history_aware_retriever(
#             llm, 
#             retriever, 
#             contextualize_q_prompt
#         )
        
#         # document chain
#         question_answer_chain = create_stuff_documents_chain(
#             llm, 
#             qa_prompt
#         )
        
#         # full RAG chain
#         rag_chain = create_retrieval_chain(
#             history_aware_retriever, 
#             question_answer_chain
#         )
        
#         result = rag_chain.invoke({
#             "input": keyword,
#             "chat_history": chat_history
#         })
        
#         answer = result['answer']
        
#         # Update session with new interaction
#         ChatSessionManager.update_session(
#             session_id, 
#             keyword, 
#             answer
#         )
        
#         return PlainTextResponse(answer)
    
#     except Exception as e:
#         error_msg = f"Error processing your request: {str(e)}. Please contact support for assistance."
#         print(f"Error in prompts_keyword: {str(e)}")
#         return PlainTextResponse(error_msg, status_code=500)

# @app.get("/api/artificial-intelligence/chathistory")
# async def get_session_history(session_id: str):
#     """Retrieve session history"""
#     try:
#         session = ChatSessionManager._sessions.get(session_id)
#         if not session:
#             raise HTTPException(status_code=404, detail="Session not found")
        
#         return {
#             "chat_history": session.get('chat_history', []),
#             "tenantId": session.get('tenantId')
#         }
#     except Exception as e:
#         raise HTTPException(status_code=500, detail=str(e))

# @app.delete("/api/artificial-intelligence/reset-session")
# async def reset_session(session_id: str):
#     """Reset a specific conversation session"""
#     try:
#         if session_id in ChatSessionManager._sessions:
#             del ChatSessionManager._sessions[session_id]
#         return {"status": "success", "message": "Session reset successfully"}
#     except Exception as e:
#         raise HTTPException(status_code=500, detail=str(e))

In [ ]:
@app.post("/api/artificial-intelligence/links")
async def upload_web_urls(
    tenantId: str,
    urls: List[str] = Form(...)):
    """
    Process and store web URL content
    Args:
        tenantId: Identifier for the tenant
        urls: List of URLs to process   
    Returns:
        JSON response indicating processing status   
    Raises:
        HTTPException: When no valid URLs are provided
    """
    all_docs = []  
    for url in urls:
        try:
            loader = WebBaseLoader(url)
            docs = loader.load()  
            for doc in docs:
                doc.metadata["tenantId"] = tenantId
                doc.metadata["source"] = url
            all_docs.extend(docs)
        except Exception as e:
            print(f"Error loading URL {url}: {e}")
            continue
    if not all_docs:
        raise HTTPException(status_code=400, detail="No valid URLs provided or failed to fetch content.")
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.0)
    chunk_size = 500
    chunk_overlap = 50
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    for doc in all_docs:
        curr_split_docs = text_splitter.split_documents([doc])
        document_id = str(uuid.uuid4())
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs.extend(curr_split_docs)
        chunk_ids.extend(curr_chunk_ids)
        upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": tenantId}
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    return {"status": "success", "message": "URLs processed and stored successfully."}
############################# links@edit #############################
@app.put("/api/artificial-intelligence/links")
async def upload_web_urls(
    tenantId: str,
    embeddedId: str,
    urls: List[str] = Form(...)):
    """
    Update web URL content for a specific embeddedId.
    
    Args:
        tenantId: Identifier for the tenant
        embeddedId: The ID of the document to update
        urls: List of URLs to be updated
    
    Returns:
        JSON response indicating processing status
    
    Raises:
        HTTPException: When no valid URLs are provided or if the embeddedId does not exist.
    """
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    
    if embeddedId not in upload_documents:
        raise HTTPException(status_code=404, detail="The embeddedId does not exist.")
    
    try:
        all_docs = []
        for url in urls:
            try:
                loader = WebBaseLoader(url)
                docs = loader.load()  
                for doc in docs:
                    doc.metadata["tenantId"] = tenantId
                    doc.metadata["source"] = url
                all_docs.extend(docs)
            except Exception as e:
                print(f"Error loading URL {url}: {e}")
                continue
        
        if not all_docs:
            raise HTTPException(status_code=400, detail="No valid URLs provided or failed to fetch content.")
        
        embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
        vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
        llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.0)
        chunk_size = 500
        chunk_overlap = 50
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        split_docs = []
        chunk_ids = []
        
        existing_chunk_ids = [key for key in upload_documents if upload_documents[key]['id'] == embeddedId]
        
        for chunk_id in existing_chunk_ids:
            del upload_documents[chunk_id]  
        
        for doc in all_docs:
            curr_split_docs = text_splitter.split_documents([doc])
            document_id = embeddedId 
            curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
            split_docs.extend(curr_split_docs)
            chunk_ids.extend(curr_chunk_ids)
            upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": tenantId}
        
        vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
        with open(uploaded_documents_path, "w") as f:
            json.dump(upload_documents, f, indent=4)
        
        return {
            "status": "success",
            "message": "URLs updated successfully.",
            "embeddedId": embeddedId,
            "updated_urls": urls
        }
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error updating URLs: {str(e)}")

In [ ]:
import os, tempfile
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import PyPDFLoader

# Function to summarize a document
def summarize_document(openai_api_key, pinecone_api_key, pinecone_index, source_doc):
    try:
        # Validate inputs
        if not openai_api_key or not pinecone_api_key or not pinecone_index or not source_doc:
            raise ValueError("Missing required fields or file.")

        # Set environment variables for API keys
        os.environ['OPENAI_API_KEY'] = openai_api_key
        os.environ['PINECONE_API_KEY'] = pinecone_api_key

        # Save uploaded file temporarily to disk, load and split the file into pages, delete temp file
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            tmp_file.write(source_doc.read())
        loader = PyPDFLoader(tmp_file.name)
        pages = loader.load_and_split()
        os.remove(tmp_file.name)

        # Create embeddings for the pages and insert into Pinecone vector database
        embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
        vectorstore = PineconeVectorStore.from_documents(pages, embeddings, index_name=pinecone_index)

        # Initialize the ChatOpenAI module, load and run the summarize chain
        llm = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)
        chain = load_summarize_chain(llm, chain_type="stuff")
        search = vectorstore.similarity_search(" ")
        summary = chain.run(input_documents=search, question="Write a concise summary within 200 words.")

        return summary
    except Exception as e:
        return f"An error occurred: {e}"


In [ ]:

############################ Summarizer #################################################
# class SummarizationRequest(BaseModel):
#     caseId: str
#     onBehalfOf: str
#     notes: List[Any]

# @app.post("/api/artificial-intelligence/summarize")
# async def summarize(request: SummarizationRequest):
#     """Combine JSON upload and summary generation into one endpoint."""
#     try:
#         json_data = {"caseId": request.caseId, "notes": request.notes}
#         text = " ".join([str(note) if isinstance(note, str) else json.dumps(note) for note in request.notes])
#         word_count = len(text.split())
#         if word_count <= 200:
#             max_length = 50  
#         elif word_count <= 1000:
#             max_length = 150  
#         else:
#             max_length = 300 
        
#         json_doc = Document(page_content=json.dumps(json_data), metadata={'fileName': 'query_json'})
#         split_docs = [json_doc]
#         print(f"Processing {len(split_docs)} documents.")
        
#         llm = ChatOpenAI(
#             model="gpt-3.5-turbo",
#             #model='gpt-4o-mini',
#             temperature=0.0,
#             max_tokens=max_length,
#             top_p=1.0,
#         )
        
#         patientPrompt = """ Write a detailed and structured summary of the following text:
#         "{split_docs}"
#          These are the conversations that might have happened between the advocates or patients. These notes are not directly sent to me.
#         summarize all the notes as if im a patient and address patient as me and include minor details also.
#         Be sure to highlight important note, medical details and Recommendations.
#         Include the major events in a bullet points if available.

#         Complete the sentence at the end with a full stop.

#                 """

#         prompt_template = """Write a detailed and structured summary of the following text:
#         "{split_docs}"
#         Please summarize the content above by addressing the following aspects in an integrated narrative format:
#         - Introduce the patient, including their full name (if mentioned), age, location, relevant medical history, and current condition at the time of documentation.
#         - Document the involvement of family members and advocates in the patient's care, specifying their names, roles, and any significant communications or actions they have taken.
#         - Provide a comprehensive account of the patient's treatment history, detailing specific challenges (e.g., mental status changes, medication side effects), responses from the medical team, and any milestones or setbacks in the patient's progress.
#         - Highlight the collaboration between medical professionals, family members, and advocates, noting specific meetings, consultations, and shared decisions made to manage the patient's care.
#         - Include updates on the case status (e.g., "IN-PROGRESS," "DISCHARGED") as well as any recommendations or plans for ongoing care, discharge planning, or long-term health considerations.

#         Ensure the summary is:
#         - **Comprehensive**: Cover all necessary details to serve as a formal record of the patient's care.
#         - **Well-Organized**: Present the information in a logical flow without using section headers, maintaining clarity and readability.
#         - **Professional in Tone**: Write in a factual, formal tone suitable for medical or administrative reporting.
#         - **Clear and Concise**: Use straightforward language that is easy to understand.
#         - **Structured**: Format the summary in paragraph form for readability, ensuring logical flow between points.

#         Additionally, please conclude with any recommendations for next steps or ongoing care considerations that were mentioned in the notes.
#         The tone should be professional yet empathetic, reflecting the seriousness of the patient's condition while acknowledging the efforts of all parties involved in their care.
#         Conclude the summary with a recap of any critical points or follow-up actions necessary to ensure continuity of care.

#         Complete the sentence at the end with a full stop.

#         """
        
#         if(request.onBehalfOf == "patient"):
#             prompt_template = patientPrompt
        
#         prompt = PromptTemplate.from_template(prompt_template)
#         llm_chain = LLMChain(llm=llm, prompt=prompt)
#         stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="split_docs")
#         summary = stuff_chain.run(split_docs)
#         if not summary.strip().endswith('.'):
#             summary += '.'
#         summary = summary.replace("\n", " ")
#         return JSONResponse(content={
#             "summary": summary
#         })
#     except Exception as e:
#         raise HTTPException(status_code=500, detail=f"Error during summarizing the documents: {str(e)}") 
    

In [ ]:
# class UploadLinkRequest(BaseModel):
#     tenantId: str
#     url: str 
# @app.post("/api/artificial-intelligence/links")
# async def upload_web_url(request: UploadLinkRequest, background_tasks: BackgroundTasks):

#     """
#     Process and store web URL content.
    
#     Args:
#         request: UploadLinkRequest containing tenantId and url.
    
#     Returns:
#         JSON response indicating processing status.
    
#     Raises:
#         HTTPException: When no valid url are provided.
#     """
#     all_docs = []  
#     url = request.url.split(",")  
#     for url in url:
#         try:
#             loader = WebBaseLoader(url.strip())  
#             docs = loader.load()  
#             for doc in docs:
#                 doc.metadata["tenantId"] = request.tenantId
#                 doc.metadata["source"] = url.strip()
#             all_docs.extend(docs)
#         except Exception as e:
#             print(f"Error loading URL {url}: {e}")
#             continue
#     if not all_docs:
#         raise HTTPException(status_code=400, detail="No valid url provided or failed to fetch content.")
    
#     embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
#     vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
#     llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
#     chunk_size = 2000
#     chunk_overlap = 500
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#     split_docs = []
#     chunk_ids = []
#     uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    
#     if not os.path.exists(uploaded_documents_path):
#         with open(uploaded_documents_path, "w") as f:
#             json.dump({}, f)
    
#     with open(uploaded_documents_path, "r") as f:
#         upload_documents = json.load(f)
    
#     document_id =  str(uuid.uuid4())
#     for doc in all_docs:
#         curr_split_docs = text_splitter.split_documents([doc])
#         document_id = document_id
#         curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
#         split_docs.extend(curr_split_docs)
#         chunk_ids.extend(curr_chunk_ids)
#         upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": request.tenantId}
    
#     vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
    
#     with open(uploaded_documents_path, "w") as f:
#         json.dump(upload_documents, f, indent=4)
    
#     return {"status": "success", "message": "url processed and stored successfully.", "data": {"documentId": document_id}}



# import os
# import json
# import uuid
# from fastapi import FastAPI, HTTPException, BackgroundTasks
# from pydantic import BaseModel
# from langchain_community.document_loaders import WebBaseLoader
# from langchain.embeddings import OpenAIEmbeddings
# from langchain_pinecone import PineconeVectorStore
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from PyPDF2 import PdfReader
# import pdfkit  
# import requests  


# class UploadLinkRequest(BaseModel):
#     tenantId: str
#     url: str 

# @app.post("/api/artificial-intelligence/links")
# async def upload_web_url(request: UploadLinkRequest, background_tasks: BackgroundTasks):
#     """
#     Process and store web URL content.
    
#     Args:
#         request: UploadLinkRequest containing tenantId and url.
    
#     Returns:
#         JSON response indicating processing status.
    
#     Raises:
#         HTTPException: When no valid url are provided.
#     """
#     all_docs = []  
#     urls = request.url.split(",")  # Split the input into a list of URLs
    
#     for url in urls:
#         url = url.strip()
#         try:
#             if url.lower().endswith('.pdf'):
#                 # If the URL is a PDF, extract text from the PDF
#                 response = requests.get(url)
#                 pdf_reader = PdfReader(response.content)
#                 extracted_text = ""
#                 for page in pdf_reader.pages:
#                     extracted_text += page.extract_text() + "\n"
                
#                 # Create a Document object with the extracted text
#                 doc = Document(page_content=extracted_text, metadata={"source": url, "tenantId": request.tenantId})
#                 all_docs.append(doc)
#             else:
#                 # If the URL is not a PDF, load it normally
#                 loader = WebBaseLoader(url)
#                 docs = loader.load()
#                 for doc in docs:
#                     doc.metadata["tenantId"] = request.tenantId
#                     doc.metadata["source"] = url
#                     all_docs.append(doc)
#         except Exception as e:
#             print(f"Error loading URL {url}: {e}")
#             continue

#     if not all_docs:
#         raise HTTPException(status_code=400, detail="No valid URL provided or failed to fetch content.")
    
#     embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
#     vectorstore = PineconeVectorStore(index_name=os.getenv("PINECONE_INDEX_NAME"), embedding=embeddings)
#     chunk_size = 800
#     chunk_overlap = 150
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#     split_docs = []
#     chunk_ids = []
#     uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    
#     if not os.path.exists(uploaded_documents_path):
#         with open(uploaded_documents_path, "w") as f:
#             json.dump({}, f)
    
#     with open(uploaded_documents_path, "r") as f:
#         upload_documents = json.load(f)
    
#     document_id = str(uuid.uuid4())
#     for doc in all_docs:
#         curr_split_docs = text_splitter.split_documents([doc])
#         curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
#         split_docs.extend(curr_split_docs)
#         chunk_ids.extend(curr_chunk_ids)
#         upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": request.tenantId}
    
#     vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
    
#     with open(uploaded_documents_path, "w") as f:
#         json.dump(upload_documents, f, indent=4)
    
#     return {"status": "success", "message": "URL processed and stored successfully.", "data": {"documentId": document_id}}


# class UploadLinkRequest(BaseModel):
#     tenantId: str
#     url: str 
# @app.post("/api/artificial-intelligence/links")
# async def upload_web_url(request: UploadLinkRequest, background_tasks: BackgroundTasks):

#     """
#     Process and store web URL content.
    
#     Args:
#         request: UploadLinkRequest containing tenantId and url.
    
#     Returns:
#         JSON response indicating processing status.
    
#     Raises:
#         HTTPException: When no valid url are provided.
#     """
#     all_docs = []  
#     url = request.url.split(",")  
#     for url in url:
#         try:
#             loader = WebBaseLoader(url.strip())  
#             docs = loader.load()  
#             for doc in docs:
#                 doc.metadata["tenantId"] = request.tenantId
#                 doc.metadata["source"] = url.strip()
#             all_docs.extend(docs)
#         except Exception as e:
#             print(f"Error loading URL {url}: {e}")
#             continue
#     if not all_docs:
#         raise HTTPException(status_code=400, detail="No valid url provided or failed to fetch content.")
    
#     embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
#     vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
#     llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
#     chunk_size = 300
#     chunk_overlap = 50
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
#     split_docs = []
#     chunk_ids = []
#     uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    
#     if not os.path.exists(uploaded_documents_path):
#         with open(uploaded_documents_path, "w") as f:
#             json.dump({}, f)
    
#     with open(uploaded_documents_path, "r") as f:
#         upload_documents = json.load(f)
    
#     document_id =  str(uuid.uuid4())
#     for doc in all_docs:
#         curr_split_docs = text_splitter.split_documents([doc])
#         document_id = document_id
#         curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
#         split_docs.extend(curr_split_docs)
#         chunk_ids.extend(curr_chunk_ids)
#         upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": request.tenantId}
    
#     vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
    
#     with open(uploaded_documents_path, "w") as f:
#         json.dump(upload_documents, f, indent=4)
    
#     return {"status": "success", "message": "url processed and stored successfully.", "data": {"documentId": document_id}}



from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
#from langserve import add_routes
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.messages.base import BaseMessage
from fastapi import UploadFile, File, HTTPException
from langchain_openai import OpenAIEmbeddings
from langchain.chains import LLMChain
from typing import Any
import uuid
import pylibmagic
from fastapi import Query
import io
import docx
import shutil
import uvicorn
from typing import List, Dict
import numpy as np
import os
import os
import json
import uuid
import asyncio
from fastapi import FastAPI, HTTPException, BackgroundTasks
from pydantic import BaseModel
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from crawl4ai import AsyncWebCrawler 
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_pinecone import PineconeVectorStore
from langchain_community.retrievers import PineconeHybridSearchRetriever
from fastapi.responses import PlainTextResponse
from pinecone import Pinecone, ServerlessSpec
from langchain_core.runnables import RunnableParallel
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable
from pydantic import BaseModel, Field, validator, ValidationError
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from tqdm.auto import tqdm
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from dotenv import load_dotenv
from fastapi.responses import RedirectResponse
import glob
from fastapi import  UploadFile
from fastapi.responses import JSONResponse
from uuid import uuid4
import time
from fastapi.middleware.cors import CORSMiddleware
from fastapi import  Request
from fastapi import  BackgroundTasks, UploadFile, Form
import getpass
import os
import concurrent.futures
from fastapi import  UploadFile
from PyPDF2 import PdfReader  
import docx
import json

In [ ]:
 patientPrompt = """ Write a detailed and structured summary of the following text:
        "{split_docs}"
        These are the conversations that might have happened between the advocates or patients. These notes are not directly sent to me.
        Summarize all the notes as if you are addressing me, the patient, and include minor details as well.
        Be sure to highlight important notes, medical details, and recommendations.
        Include the major events in bullet points if available.

        Complete the sentence at the end with a full stop.
        """

        prompt_template = f"""Write a detailed and structured summary of the following text:
        "{{split_docs}}"

        Please summarize the content above by addressing the following aspects in an integrated narrative format:
        - Introduce the patient, age, location, relevant medical history, and current condition.

        - No need to mention case ID
        - No need to talk about note accessType
        
        Ensure the summary is:
        - **Comprehensive**: Cover all necessary details to serve as a formal record of the patient's care.
        - **Well-Organized**: Present the information in a logical flow without using section headers, maintaining clarity and readability.
        - **Professional in Tone**: Write in a factual, formal tone suitable for medical or administrative reporting.
        - **Clear and Concise**: Use straightforward language that is easy to understand.
        - **Structured**: Format the summary in paragraph form for readability, ensuring logical flow between points.

        Additionally, please conclude with any recommendations for next steps or ongoing care considerations that were mentioned in the notes.
        The tone should be professional yet empathetic, reflecting the seriousness of the patient's condition while acknowledging the efforts of all parties involved in their care.
        Conclude the summary with a recap of any critical points or follow-up actions necessary to ensure continuity of care.

        Complete the sentence at the end with a full stop.
        """

## WebaseLoder

In [10]:
%pip install -qU langchain_community beautifulsoup4



Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.noveltytechnology.com/about-us")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [12]:
docs = loader.load()

docs[0]

Document(metadata={'source': 'https://www.noveltytechnology.com/about-us', 'title': 'About Us | Novelty Technology', 'description': 'Discover Novelty Technology, a leader in innovative IT solutions. We are dedicated to delivering tailored services that enhance efficiency and empower your business.', 'language': 'en'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAbout Us | Novelty Technology\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ntop of page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSolutionsProcessProjectsAbout UsAbout UsDriven by Excellence,\nFueled by PassionYour Success, Our PurposeWe’re a group of professionals who thrive on designing and building solutions that make a real impact. Our collective expertise and commitment to excellence ensure that every project we undertake is a perfect blend of creativity, precision, and purpose. Your success is at the heart of everything we do becaus

In [13]:
print(docs[0].metadata)


{'source': 'https://www.noveltytechnology.com/about-us', 'title': 'About Us | Novelty Technology', 'description': 'Discover Novelty Technology, a leader in innovative IT solutions. We are dedicated to delivering tailored services that enhance efficiency and empower your business.', 'language': 'en'}


In [14]:
%pip install -qU  nest_asyncio

# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

Note: you may need to restart the kernel to use updated packages.


In [15]:
loader = WebBaseLoader("https://www.noveltytechnology.com/about-us")
loader.requests_per_second = 1
docs = loader.aload()
docs

Fetching pages: 100%|##########| 1/1 [00:03<00:00,  3.07s/it]
/Users/ujjwal/Library/Python/3.9/lib/python/site-packages/bs4/builder/__init__.py:314: RuntimeWarning: coroutine 'AsyncChromiumLoader.ascrape_playwright' was never awaited
  for attr in list(attrs.keys()):


[Document(metadata={'source': 'https://www.noveltytechnology.com/about-us', 'title': 'About Us | Novelty Technology', 'description': 'Discover Novelty Technology, a leader in innovative IT solutions. We are dedicated to delivering tailored services that enhance efficiency and empower your business.', 'language': 'en'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAbout Us | Novelty Technology\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ntop of page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSolutionsProcessProjectsAbout UsAbout UsDriven by Excellence,\nFueled by PassionYour Success, Our PurposeWe’re a group of professionals who thrive on designing and building solutions that make a real impact. Our collective expertise and commitment to excellence ensure that every project we undertake is a perfect blend of creativity, precision, and purpose. Your success is at the heart of everything we do becau

## Unstructured URL Loader

In [1]:
%pip install --upgrade --quiet unstructured



Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import UnstructuredURLLoader

urls = [
    "https://www.noveltytechnology.com/about-us",
    "https://www.noveltytechnology.com/solutions",
]

/Users/ujjwal/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
loader = UnstructuredURLLoader(urls=urls)

data = loader.load()

data[0]

Document(metadata={'source': 'https://www.noveltytechnology.com/about-us'}, page_content="About Us\n\nDriven by Excellence, Fueled by Passion\n\nYour Success, Our Purpose\n\nWe’re a group of professionals who thrive on designing and building solutions that make a real impact. Our collective expertise and commitment to excellence ensure that every project we undertake is a perfect blend of creativity, precision, and purpose. Your success is at the heart of everything we do because when you succeed, we succeed.\n\nBook a Consultation\n\nOur Philosophy\n\nSimplicity\n\nWe design software with a clean, intuitive interface, making complex tasks feel effortless. Simplicity drives efficiency, helping your team work smarter, not harder.\n\nSecurity\n\nSecurity is non-negotiable in our software. We implement advanced protections to keep your data secure, giving you confidence and peace of mind.\n\nScalibility\n\nOur software scales with your business, adapting to increased demand and evolving n

## Selenium

In [4]:
%pip install --upgrade --quiet selenium unstructured

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_community.document_loaders import SeleniumURLLoader

urls = [
    "https://www.noveltytechnology.com/about-us",
    "https://www.noveltytechnology.com/solutions",
]

loader = SeleniumURLLoader(urls=urls)

data = loader.load()

data[0]

Document(metadata={'source': 'https://www.noveltytechnology.com/about-us', 'title': 'About Us | Novelty Technology', 'description': 'Discover Novelty Technology, a leader in innovative IT solutions. We are dedicated to delivering tailored services that enhance efficiency and empower your business.', 'language': 'en'}, page_content="About Us\n\nDriven by Excellence, Fueled by Passion\n\nYour Success, Our Purpose\n\nWe’re a group of professionals who thrive on designing and building solutions that make a real impact. Our collective expertise and commitment to excellence ensure that every project we undertake is a perfect blend of creativity, precision, and purpose. Your success is at the heart of everything we do because when you succeed, we succeed.\n\nBook a Consultation\n\nOur Philosophy\n\nSimplicity\n\nWe design software with a clean, intuitive interface, making complex tasks feel effortless. Simplicity drives efficiency, helping your team work smarter, not harder.\n\nSecurity\n\nSe

## Crawl4AI

In [ ]:
class UploadLinkRequest(BaseModel):
    tenantId: str
    url: str 

@app.post("/api/artificial-intelligence/links")
async def upload_web_url(request: UploadLinkRequest, background_tasks: BackgroundTasks):
    """
    Process and store web URL content.
    
    Args:
        request: UploadLinkRequest containing tenantId and url.
    
    Returns:
        JSON response indicating processing status.
    
    Raises:
        HTTPException: When no valid url are provided.
    """
    all_docs = []  
    urls = request.url.split(",")  

    for url in urls:
        url = url.strip()
        try:
            async with AsyncWebCrawler(verbose=True) as crawler:
                result = await crawler.arun(url=url)  # Running the crawler on the URL
                extracted_content = result.markdown  # Extracted content in markdown format
                doc = Document(page_content=extracted_content, metadata={"source": url, "tenantId": request.tenantId})
                all_docs.append(doc)

        except Exception as e:
            print(f"Error loading URL {url}: {e}")
            continue

    if not all_docs:
        raise HTTPException(status_code=400, detail="No valid URL provided or failed to fetch content.")
    
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=os.getenv("PINECONE_INDEX_NAME"), embedding=embeddings)
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
    chunk_size = 3500
    chunk_overlap = 1000
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_docs = []
    chunk_ids = []
    uploaded_documents_path = "do_not_delete_uploaded_documents.json"
    
    if not os.path.exists(uploaded_documents_path):
        with open(uploaded_documents_path, "w") as f:
            json.dump({}, f)
    
    with open(uploaded_documents_path, "r") as f:
        upload_documents = json.load(f)
    
    document_id = str(uuid.uuid4())
    for doc in all_docs:
        curr_split_docs = text_splitter.split_documents([doc])
        curr_chunk_ids = [f"{document_id}_chunk_{i+1}" for i in range(len(curr_split_docs))]
        split_docs.extend(curr_split_docs)
        chunk_ids.extend(curr_chunk_ids)
        upload_documents[document_id] = {"source": doc.metadata["source"], "id": document_id, "tenantId": request.tenantId}
    
    vectorstore.add_documents(documents=split_docs, ids=chunk_ids)
    
    with open(uploaded_documents_path, "w") as f:
        json.dump(upload_documents, f, indent=4)
    
    return {"status": "success", "message": "URL processed and stored successfully.", "data": {"documentId": document_id}}

In [8]:
!pip install -U crawl4ai


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 16.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 9.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.4 MB/s eta 0:00:00
  Created wheel for pytest-mockito: filename=pytest_mockito-0.0.4-py3-none-any.whl size=3703 sha256=99d564fc4150974112f8155792c46db269de9ab0a10fcf2b07da7fbf6ae069ad
  Stored in directory: /Users/ujjwal/Library/Caches/pip/wheels/37/ef/57/49fa3483387a8257932046830ec1cbfd46744531882bf2f6f6
Successfully built pytest-mockito
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.23.4
    Uninstalling pydantic_core-2.23.4:
      Successfully uninstalled pydantic_core-2.23.4
  Attempting uninstall: psutil
    Found existing installation: psutil 6.1.0
    Uninstalling psutil-6.1.0:
      Successfully 

In [10]:
!crawl4ai-setup

/Users/ujjwal/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[INIT].... → Running post-installation setup...
[INIT].... → Installing Playwright browsers...
[COMPLETE] ● Playwright installation completed successfully.
[INIT].... → Starting database initialization...
[COMPLETE] ● Database backup created at: /Users/ujjwal/.crawl4ai/crawl4ai.db.backup_20250104_203017
[INIT].... → Starting database migration...
[COMPLETE] ● Migration completed. 0 records processed.
[COMPLETE] ● Database initialization completed successfully.
[COMPLETE] ● Post-installation setup completed!


In [11]:
!crawl4ai-doctor

/Users/ujjwal/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[INIT].... → Running Crawl4AI health check...
[INIT].... → Crawl4AI 0.4.246
[TEST].... ℹ Testing crawling capabilities...
[EXPORT].. ℹ Exporting PDF and taking screenshot took 0.34s
[FETCH]... ↓ https://crawl4ai.com... | Status: True | Time: 5.57s
[SCRAPE].. ◆ Processed https://crawl4ai.com... | Time: 34ms
[COMPLETE] ● https://crawl4ai.com... | Status: True | Total: 5.61s
[COMPLETE] ● ✅ Crawling test passed!


In [17]:
import asyncio
from crawl4ai import AsyncWebCrawler

async def main():
    # Create an instance of AsyncWebCrawler
    async with AsyncWebCrawler(verbose=True) as crawler:
        # Run the crawler on a URL
        result = await crawler.arun(url="https://www.noveltytechnology.com/about-us")

        # Print the extracted content
        print(result.markdown)

# Check if there's an existing event loop
try:
    # For Python 3.7 and later
    loop = asyncio.get_running_loop()
except RuntimeError:
    # No event loop is running
    loop = None

if loop and loop.is_running():
    # If an event loop is already running, use it to run the main coroutine
    print("Using existing event loop.")
    task = loop.create_task(main())
else:
    # If no event loop is running, create a new one
    print("Creating a new event loop.")
    asyncio.run(main())


Using existing event loop.


[INIT].... → Crawl4AI 0.4.246
[FETCH]... ↓ https://www.noveltytechnology.com/about-us... | Status: True | Time: 1.89s
[SCRAPE].. ◆ Processed https://www.noveltytechnology.com/about-us... | Time: 44ms
[COMPLETE] ● https://www.noveltytechnology.com/about-us... | Status: True | Total: 1.98s
top of page
[](https://www.noveltytechnology.com/<https:/www.noveltytechnology.com>)
  * [Solutions](https://www.noveltytechnology.com/<https:/www.noveltytechnology.com/solutions>)
  * [Process](https://www.noveltytechnology.com/<https:/www.noveltytechnology.com/process>)
  * [Projects](https://www.noveltytechnology.com/<https:/www.noveltytechnology.com/projects>)
  * [About Us](https://www.noveltytechnology.com/<https:/www.noveltytechnology.com/about-us>)


About Us
## Driven by Excellence, Fueled by Passion
### Your Success, Our Purpose
We’re a group of professionals who thrive on designing and building solutions that make a real impact. Our collective expertise and commitment to excellence ensure th

In [35]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
from crewai_tools import ScrapeWebsiteTool
import os 
from dotenv import load_dotenv

AttributeError: Config

In [ ]:

############################# prompts #############################
def initializeVectorStore():
    """Initialize and configure vector store with embeddings
    Returns:
        Configured PineconeVectorStore instance
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectorstore = PineconeVectorStore(index_name=pinecone_index_name, embedding=embeddings)
    return vectorstore

llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature=0.0
)
@app.get("/api/artificial-intelligence/prompts")
async def prompts_keyword(tenantId: str = Query(...), keyword: str = Query(...)):
    """Process keyword prompts and retrieve answers using improved RAG chain
    Args:
        tenantId: Identifier for the tenant
        keyword: Search keyword/prompt  
    Returns:
        PlainText response with answer or error message
    """
    try:
        print(tenantId, keyword)
        vectorstore = initializeVectorStore()
        template = """Answer the question based only on the following context:
        {context}
        User Question:
        {question}
         If the user greets you, respond with one of the following without searching information from knowledge base:
        - "Hello! How can I assist you today?"
        - "Hi there! What can I do for you?"
        - "Good day! How can I assist you?"
        - "Hey! What can I help you with?"
        - "Hi! How can I support you today?"
        - "Welcome! How may I help you?"
        - "Salutations! How can I assist you?"
        - "What's up? How can I help?"
        - "Good to see you! How can I assist?"

        If the question asked by user is not exact match to the knowledge base question then say: "I don't have the information you're looking for, please provide additional details." And if it's greetings then give greetings response.    
        Only give answer if you're 100 per cent sure. If you're not sure then say: "I don't have the information you're looking for, please provide additional details."      
        
        """

        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={
                "k": 1,
                "filter": {
                    'tenantId': {'$eq': tenantId}
                }
            }
        )
        prompt = ChatPromptTemplate.from_template(template)
        rag_chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser())
        answer = rag_chain.invoke(keyword)
        return PlainTextResponse(answer)
    except Exception as e:
        error_msg = f"Error processing your request: {str(e)}. Please contact support for assistance."
        print(f"Error in prompts_keyword: {str(e)}") 
        return PlainTextResponse(error_msg, status_code=500)